In [2]:
#importing liberaries
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
import pathlib
from keras.layers import BatchNormalization
from tensorflow.keras import Sequential
import tensorflow_datasets as tfds
import os
import cv2
import matplotlib.pyplot as plot
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam
from os import listdir
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [3]:
EPOCHS = 30
INIT_LR = 1e-1
Batch_size =32
Image_size = (128,128)
directory_root = '/kaggle/input/flowers-recognition/flowers'
default_image_size = tuple((128, 128))

In [4]:
files = os.listdir(directory_root)
print(files)

['dandelion', 'daisy', 'sunflower', 'tulip', 'rose']


In [7]:
#creating image list containing all Dataset images as directories
image_dir_list = []
label_list = []
root_dir = listdir(directory_root)
for plant_disease_folder in root_dir:
    plant_disease_dir = listdir(f"{directory_root}/{plant_disease_folder}")
    for image in plant_disease_dir:
        plant_image_dir = f"{directory_root}/{plant_disease_folder}/{image}"
        if plant_image_dir.endswith(".jpg") == True or plant_image_dir.endswith(".JPG") == True:
            image_dir_list.append(plant_image_dir)
            label_list.append(plant_disease_folder)

In [8]:
#importing all images in a list
images_list= []
for image_dir in image_dir_list:
    if image_dir.endswith(".jpg") == True or image_dir.endswith(".JPG") == True:
        image = cv2.imread(image_dir)
        image = cv2.resize(image, default_image_size)   
        image_array = img_to_array(image)
        images_list.append(image_array)

In [9]:
#Converting images to numpy array with normalization
np_image_list = np.array(images_list, dtype=np.float16) / 225.0
print(np_image_list.shape)

(4317, 128, 128, 3)


In [10]:
#binarizing labels
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
n_classes = len(label_binarizer.classes_)

In [11]:
X = np_image_list
y = image_labels

In [12]:
#test lists
accuracy_test = []
#Train lists
accuracy_train = []

In [15]:
from sklearn.model_selection import KFold
k = 3

kf = KFold(n_splits= k, random_state=True, shuffle=True)


for train_index, test_index in kf.split(X) :
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    np.random.seed(1000)

    #Instantiation
    AlexNet = Sequential()

    #1st Convolutional Layer
    AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding ='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #2nd Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #3rd Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

    #4th Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))

    #5th Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #Passing it to a Fully Connected layer
    AlexNet.add(Flatten())
    # 1st Fully Connected Layer
    AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    AlexNet.add(Dropout(0.4))

    #2nd Fully Connected Layer
    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

   #3rd Fully Connected Layer
    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #Output Layer
    AlexNet.add(Dense(5))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))
    AlexNet.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0003), loss='binary_crossentropy', metrics=["accuracy"])
   
    #training model
    
    AlexNet.fit(X_train,y_train,epochs= 30)

    
    #model prediction 
  
    Model_pred_train = AlexNet.predict(X_train)
  
    Model_pred_test =  AlexNet.predict(X_test)

    
    Model_pred_test = Model_pred_test > 0.5
    Model_pred_train = Model_pred_train > 0.5
    
 
    ################################################33
    print("Test Evaluation")
    print("="*40)
    #Calculating Test accuracy
    print('Test accuracy is :' + str(accuracy_score(y_test,Model_pred_test)*100))
    accuracy = accuracy_score(y_test,Model_pred_test)*100
    accuracy_test.append(accuracy)
    ################################################################
    print("="*40)
    print("Train Evaluation")
    print("="*40)
    print('Training accuracy is : ' + str(accuracy_score(y_train,Model_pred_train)*100))
    t_accuracy = accuracy_score(y_train,Model_pred_train)*100
    accuracy_train.append(t_accuracy)  
    #################################################################
    print("="*40)
    

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
90/90 [==============================] - 4s 22ms/step - loss: 0.6234 - accuracy: 0.4910
Epoch 2/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5772 - accuracy: 0.5709
Epoch 3/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5438 - accuracy: 0.6240
Epoch 4/30
90/90 [==============================] - 2s 22ms/step - loss: 0.5203 - accuracy: 0.6685
Epoch 5/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5022 - accuracy: 0.6862
Epoch 6/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4783 - accuracy: 0.7140
Epoch 7/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4527 - accuracy: 0.7613
Epoch 8/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4334 - accuracy: 0.7874
Epoch 9/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4129 - accuracy: 0.8138
Epoch 10/30
90/90 [==============================] - 2s 22ms/step - loss: 0.3880 - accuracy: 0.8471
Epoch 11/

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
90/90 [==============================] - 4s 21ms/step - loss: 0.6223 - accuracy: 0.5014
Epoch 2/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5742 - accuracy: 0.5876
Epoch 3/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5432 - accuracy: 0.6289
Epoch 4/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5144 - accuracy: 0.6734
Epoch 5/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4951 - accuracy: 0.7008
Epoch 6/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4721 - accuracy: 0.7220
Epoch 7/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4526 - accuracy: 0.7655
Epoch 8/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4279 - accuracy: 0.7981
Epoch 9/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4056 - accuracy: 0.8287
Epoch 10/30
90/90 [==============================] - 2s 21ms/step - loss: 0.3849 - accuracy: 0.8596
Epoch 11/

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/30
90/90 [==============================] - 4s 21ms/step - loss: 0.6233 - accuracy: 0.5042
Epoch 2/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5695 - accuracy: 0.5813
Epoch 3/30
90/90 [==============================] - 2s 21ms/step - loss: 0.5434 - accuracy: 0.6320
Epoch 4/30
90/90 [==============================] - 2s 22ms/step - loss: 0.5204 - accuracy: 0.6602
Epoch 5/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4944 - accuracy: 0.6984
Epoch 6/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4735 - accuracy: 0.7283
Epoch 7/30
90/90 [==============================] - 2s 21ms/step - loss: 0.4523 - accuracy: 0.7564
Epoch 8/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4309 - accuracy: 0.7943
Epoch 9/30
90/90 [==============================] - 2s 22ms/step - loss: 0.4091 - accuracy: 0.8242
Epoch 10/30
90/90 [==============================] - 2s 22ms/step - loss: 0.3855 - accuracy: 0.8562
Epoch 11/

# **Test Values & Averages**

In [16]:
test_accuracy_avg = 0
for accuracy in accuracy_test:
    test_accuracy_avg += accuracy
test_accuracy_average = test_accuracy_avg/k
print("Accuracy array: " + str(accuracy_test))
print("Average Accuracy : " + str(test_accuracy_average))

Accuracy array: [62.26546212647672, 64.62821403752605, 53.30090340514246, 63.72480889506602]
Average Accuracy : 81.30646282140374


# **Train Values & Averages**

In [17]:
train_accuracy_avg = 0
for train_accuracy in accuracy_train:
    train_accuracy_avg += train_accuracy
train_accuracy_average = train_accuracy_avg/k
print(accuracy_train)
print("="*40)
print("Train accuracy average: "+ str(train_accuracy_average))

[98.74913134120918, 75.88603196664351, 97.81097984711606]
Train accuracy average: 90.81538105165625
